In [1]:
from __future__ import absolute_import, division
from sklearn.decomposition import TruncatedSVD

import Model
import numpy as np 
from keras.preprocessing.sequence import pad_sequences
import senteval
import pickle
import copy
import operator

import torch
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel

PATH_SENTEVAL = '../'
PATH_TO_DATA = 'data'

maxLength = 30    
wordDim = 768    # 300  / GPT2 768 
num_neurons = [2*wordDim,2*wordDim]   # [600,600f]
extractLayer = ['X_input_second','X_input_mask_2']
kernel_reg = 0.001
batch_size = 400
merge_mode = 'concat'
checkpoint = [ "../checkpoint_mask_fre2500/Fre_2500-loss-1.3988-cata_acc-0.7049-val_loss-1.5115-cate_acc_val-0.6838_.hdf5"  ]# Change 
embeddingPath = "../Embedding/glove.840B.300d.txt"  # Change 

processedPath = "../processed/"
tokenizerFile = 'tokenizer.txt'
embeddingFile =  'embedding_matrix.txt'
dicFile = 'wordDic.txt'
marked_token = "maskedtoken"
n_component = 5

# GPT2 
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

# Glove

# print("Load embedding")
# embeddings_index = dict()
# f = open(embeddingPath  , 'r', errors = 'ignore', encoding='utf8')  # "word2vec.6B.50d.txt" for test 
# for line in f:

#     splitLine = line.split(' ')  # For 3B  splitLine = line.split()
#     word = splitLine[0]
#     coefs = np.array([float(val) for val in splitLine[1:]])
#     embeddings_index[word] = coefs
# f.close()
# print("Finish loading")

# SentEval prepare and batcher
def prepare(params, samples):
    return

def batcher(params, batch):
    global embeddings_index
    sentences = [' '.join(s) for s in batch]
            
    #sentences  = Model.preprocessing(sentences)
        
    sentenceList = []
    for sentence in sentences:
        #print(sentence)
        
        # Glove Embedding
#         print(sentence)
#         wordList = []
#         for vocab in sentence:
#             try:
#                 wordList.append(embeddings_index[vocab])
#             except:
#                 wordList.append(np.zeros(wordDim))
            
        # GPT2   
        
        wordList = []
        if len(sentence) != 0:
            #print("sentence: "+sentence)
            indexed_tokens_1 = tokenizer.encode(sentence)
            tokens_tensor_1 = torch.tensor([indexed_tokens_1])
            with torch.no_grad():
                hidden_states_1, past = model(tokens_tensor_1)
            wordList  = hidden_states_1[0].tolist()
#             output = np.mean(np.array(wordList),axis= 0, keepdims = True)
#         else: 
#             #print("sentence is empty")
#             output = np.zeros([1,wordDim])

        
      
        # SVD 
        while(len(wordList)< n_component):
            wordList.append(np.zeros(wordDim))
        
        wordList = np.array(wordList)
        
        sigma  = np.dot(wordList.T,wordList)/wordDim
        
        svd = TruncatedSVD(n_components=n_component, n_iter=5, random_state=42)
        
        svd.fit(sigma)
        for i in range(n_component):
            svd.components_[i] = svd.components_[i]*svd.singular_values_[i]
        output =  svd.components_.flatten()
        

        sentenceList.append(output)
        

    
#     take importance into account. 
#     for i in range(len(svd.singular_values_)):
#         svd.components_[i] = svd.components_[i]*svd.singular_values_[i]
    return sentenceList


/Users/chenxi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
tasks = ['CR'] # 'MR'
result = []
for task in tasks:
    print(task)
    params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 10}
    params_senteval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 64,
                                     'tenacity': 5, 'epoch_size': 4}
    # Set up logger
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    transfer_tasks =    [task]  # ['CR','MPQA','TREC','MRPC']
    # 'MR', 'CR' ,'MPQA', 'SUBJ','TREC', 'MRPC','SICKEntailment', 'SICKRelatedness'  ,'TREC','MRPC'
    results = se.eval(transfer_tasks)
    result.append(results)

CR


/Users/chenxi/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/truncated_svd.py:192: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


KeyboardInterrupt: 

In [ ]:
result

In [20]:
sentence  = 'what are you thinking'

indexed_tokens_1 = tokenizer.encode(sentence)
tokens_tensor_1 = torch.tensor([indexed_tokens_1])
with torch.no_grad():
    hidden_states_1, past = model(tokens_tensor_1)
wordList  = hidden_states_1[0].tolist()

In [22]:
wordList = np.array(wordList)

In [30]:
sigma.shape

(768, 768)

In [31]:

sigma  = np.dot(wordList.T,wordList)/wordDim
svd = TruncatedSVD(n_components=3, n_iter=5, random_state=42)

svd.fit(sigma)

TruncatedSVD(algorithm='randomized', n_components=3, n_iter=5,
       random_state=42, tol=0.0)

In [35]:
np.dot(svd.components_,sigma)

(3, 768)

In [ ]:
embeddings_index = dict()

if len(embeddings_index) == 0:
    f = open(embeddingPath  , 'r', errors = 'ignore', encoding='utf8')  # "word2vec.6B.50d.txt" for test 
    for line in f:

        splitLine = line.split(' ')  # For 3B  splitLine = line.split()
        word = splitLine[0]
        coefs = np.array([float(val) for val in splitLine[1:]])
        

    #     values = line.split()
    #     word = values[0]
    #     coefs = asarray(values[1:], dtype='float32' )
        embeddings_index[word] = coefs
    f.close()

In [ ]:
try:
    embeddings_index['third']
except:
    print("what")

In [19]:
np.mean(np.array([[2,2],[1,0]]),axis = 0 , keepdims = True).shape

(1, 2)

In [ ]:
wordSentence = ['this','is','a','test','how']
wordList = []
for vocab in wordSentence:
    wordList.append(embeddings_index[vocab])


In [ ]:
svd = TruncatedSVD(n_components=4, n_iter=5, random_state=42)
svd.fit(wordList)
output =  svd.components_.flatten()

In [ ]:
svd.components_.shape